In [1]:
import keras
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense, Flatten
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.layers.normalization import BatchNormalization
#from keras.layers.convolutional import *
import matplotlib.pyplot as plt
from keras.utils import plot_model 
from keras import models
from keras import layers
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping

/data/anaconda/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
img_width=224
img_height=224

train_data_dir="../data/train"
test_data_dir="../data/test"

batch_size=64

nb_train_samples=176021
nb_test_samples=44004

epochs = 10

In [3]:
resnet = keras.applications.resnet50.ResNet50(include_top=False, weights='imagenet', input_tensor=None, input_shape=(224,224,3), classes=1000)

model = models.Sequential()
model.add(resnet)
model.add(layers.Flatten())
model.add(Dense(2))
#model.add(Activation('relu'))
model.add(Activation('softmax'))
#for 2 axis
#model.add(Dense(2, activation=None))
#for 3 axis
#model.add(Dense(3, activation=None))
#model.add(Dense(4, activation=None))


layer_num = len(resnet.layers)

for layer in resnet.layers[:int(layer_num * 0.9)]:
        layer.trainable = False

model.load_weights('checkpoints/pathology_resnet50.02-0.29.hdf5')

for layer in resnet.layers[:int(layer_num * 0.5)]:
        layer.trainable = False
        
for layer in resnet.layers[int(layer_num * 0.5):]:
        layer.trainable = True


model_num = len(model.layers)


model.summary()
print (len(resnet.layers))
print(layer_num * 0.8)

/data/anaconda/envs/py35/lib/python3.5/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
flatten_1 (Flatten)          (None, 100352)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 200706    
_________________________________________________________________
activation_50 (Activation)   (None, 2)                 0         
Total params: 23,788,418
Trainable params: 21,562,882
Non-trainable params: 2,225,536
_________________________________________________________________
175
140.0


In [4]:
adam = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)


In [5]:
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [6]:
# Save the model according to the conditions  
checkpoint = ModelCheckpoint("checkpoints/pathology_resnet50_50percent.{epoch:02d}-{val_loss:.2f}.hdf5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')


In [7]:
# Initiate the train and test generators with data Augumentation 
train_datagen = ImageDataGenerator(
rescale = 1./255)

test_datagen = ImageDataGenerator(
rescale = 1./255)

In [8]:
train_generator = train_datagen.flow_from_directory(
train_data_dir,
target_size = (img_height, img_width),
batch_size = batch_size, 
class_mode = "categorical")

test_generator = test_datagen.flow_from_directory(
test_data_dir,
target_size = (img_height, img_width),
class_mode = "categorical")

Found 176021 images belonging to 2 classes.
Found 44004 images belonging to 2 classes.


In [ ]:
model_final = model
# Train the model 
model_final.fit_generator(
train_generator,
steps_per_epoch = nb_train_samples/batch_size,
epochs = epochs,
validation_data = test_generator,
validation_steps = nb_test_samples/batch_size,
callbacks = [checkpoint, early])

Epoch 1/10
2751/2750 [==============================] - 2758s 1s/step - loss: 0.3296 - acc: 0.9051 - val_loss: 0.3149 - val_acc: 0.9198

Epoch 00001: val_acc improved from -inf to 0.91983, saving model to checkpoints/pathology_resnet50_50percent.01-0.31.hdf5
Epoch 2/10
2751/2750 [==============================] - 2746s 998ms/step - loss: 0.2671 - acc: 0.9205 - val_loss: 0.3103 - val_acc: 0.9106

Epoch 00002: val_acc did not improve from 0.91983
Epoch 3/10
2751/2750 [==============================] - 2744s 997ms/step - loss: 0.2929 - acc: 0.9056 - val_loss: 0.2550 - val_acc: 0.9233

Epoch 00003: val_acc improved from 0.91983 to 0.92328, saving model to checkpoints/pathology_resnet50_50percent.03-0.25.hdf5
Epoch 4/10
2751/2750 [==============================] - 2743s 997ms/step - loss: 0.2858 - acc: 0.9097 - val_loss: 0.2620 - val_acc: 0.9218

Epoch 00004: val_acc did not improve from 0.92328
Epoch 5/10
2751/2750 [==============================] - 2743s 997ms/step - loss: 0.3017 - acc: 0